In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy import random
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import sys
import matplotlib
import math
from sklearn import tree
from  sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [7]:
b = np.array([1.76,54,36.23])
c = np.array([1.52,47,30.0])

numerator = np.dot(b,c)
blen = np.linalg.norm(b)
clen = np.linalg.norm(c)

cosin = numerator / (blen * clen)
cosin


0.9997393336843805

In [9]:
adngle_rad = np.arccos(cosin)
adngle_deg = adngle_rad * 360/2/np.pi
adngle_deg

1.3082463184243107

-----------------------dataset-----------------------------

In [10]:
df1 = pd.read_csv('./movies.csv')
df1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
df2 = pd.read_csv('./ratings.csv')
df2.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
df1.drop(['genres'],axis=1,inplace=True)
df2.drop(['timestamp'],axis=1,inplace=True)

In [20]:
user_item_matrix = pd.pivot_table(df2,index='movieId',columns='userId',values='rating',aggfunc='mean')
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
user_item_matrix.fillna(0,inplace=True)
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
user_votes = df2.groupby('userId')['rating'].agg('count')
movie_votes = df2.groupby('movieId')['rating'].agg('count')

user_filter = user_votes[user_votes > 50].index
movie_filter = movie_votes[movie_votes > 10].index

user_item_matrix = user_item_matrix.loc[:,user_filter]
user_item_matrix = user_item_matrix.loc[movie_filter,:]

user_item_matrix.head()

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [31]:
from scipy.sparse import csr_matrix

In [32]:
csr_data = csr_matrix(user_item_matrix.values)

In [34]:
print(csr_data[:2,:5])

  (0, 0)	4.0
  (0, 3)	4.5
  (1, 2)	4.0


In [37]:
user_item_matrix = user_item_matrix.rename_axis(None, axis=1).reset_index()
user_item_matrix.head()

,index,movieId,1,4,6,7,10,11,15,16,...,600,601,602,603,604,605,606,607,608,610
0,0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [42]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20,n_jobs=1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=1, n_neighbors=20)

In [45]:
search_word = 'Father'
recommendation = 10 # получить 10 рекомендаций по ключевому слову Father

movie_search = df1[df1['title'].str.contains(search_word)]
movie_id = movie_search.iloc[0]['movieId']
movie_id = user_item_matrix[user_item_matrix['movieId']==movie_id].index[0]
movie_id

3

In [48]:
distances, indices = knn.kneighbors(csr_data[movie_id],n_neighbors=recommendation+1) # ближайшие соседи, дистанция и индекс

In [50]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list,distances_list))
indices_distances[:5]

[(3, 6.661338147750939e-16),
 (133, 0.4611735258645464),
 (5, 0.5023407251391326),
 (46, 0.549635079548866),
 (2, 0.570618479888558)]

In [52]:
indices_distances_sorted = sorted(indices_distances,key = lambda x: x[1], reverse=False)
indices_distances_sorted

[(3, 6.661338147750939e-16),
 (133, 0.4611735258645464),
 (5, 0.5023407251391326),
 (46, 0.549635079548866),
 (2, 0.570618479888558),
 (40, 0.5824008479314086),
 (105, 0.5882489211692126),
 (203, 0.5900096405358211),
 (329, 0.5921620813935401),
 (278, 0.5933253986421428),
 (400, 0.594594318403133)]

In [53]:
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(133, 0.4611735258645464),
 (5, 0.5023407251391326),
 (46, 0.549635079548866),
 (2, 0.570618479888558),
 (40, 0.5824008479314086),
 (105, 0.5882489211692126),
 (203, 0.5900096405358211),
 (329, 0.5921620813935401),
 (278, 0.5933253986421428),
 (400, 0.594594318403133)]

In [58]:
for index_dist in indices_distances_sorted:
    movieId = user_item_matrix.iloc[index_dist[0]]['movieId']
    id = df1[df1['movieId']==movieId].index
    title = df1.iloc[id]['title'].values[0]
    print(f'{title} ---> {index_dist[1]}')


Miracle on 34th Street (1994) ---> 0.4611735258645464
Sabrina (1995) ---> 0.5023407251391326
Juror, The (1996) ---> 0.549635079548866
Grumpier Old Men (1995) ---> 0.570618479888558
Mr. Holland's Opus (1995) ---> 0.5824008479314086
Disclosure (1994) ---> 0.5882489211692126
Coneheads (1993) ---> 0.5900096405358211
Tin Cup (1996) ---> 0.5921620813935401
Sgt. Bilko (1996) ---> 0.5933253986421428
Willy Wonka & the Chocolate Factory (1971) ---> 0.594594318403133
